In [ ]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
#from torchvision import datasets, transforms
from pathlib import Path

In [ ]:
import collections
Datasets = collections.namedtuple('Datasets', ['train', 'validation', 'test'])


In [ ]:
import collections
import numpy as np
import pandas as pd
from tensorflow.python.framework import dtypes, random_seed


def load_data(data_file):
    data = pd.read_csv(data_file)
    pixels = data['pixels'].tolist()
    width = 48
    height = 48
    faces = []
    for pixel_sequence in pixels:
        # 从csv中获取人脸的数据
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        # 把脸的数据变为48*48像素，利用plt.imshow即可打印出图片
        face = np.asarray(face).reshape(width, height)
        faces.append(face)
    # 把faces从列表变为三维矩阵。(35887,)----->(35887,48,48)
    faces = np.asarray(faces)
    # 添加维度，将faces从(35887,48,48)------>(35887,48,48,1)
    faces = np.expand_dims(faces, -1)
    # one-hot编码，把属于该类表情置1，其余为0，并转换为矩阵
    emotions = pd.get_dummies(data['emotion']).to_numpy()
    return faces, emotions


class DataSet(object):
    def __init__(self, images, labels, reshape=True, dtype=dtypes.float32, seed=None):
        seed1, seed2 = random_seed.get_seed(seed)
        np.random.seed(seed1 if seed is None else seed2)
        if reshape:
            # 将images(35887,48,48,1)变为(35887,2304)
            assert images.shape[3] == 1
            images = images.reshape(images.shape[0],images.shape[1]*images.shape[2])

        # 类型转换，并进行灰度处理
        if dtype == dtypes.float32:
            images = images.astype(np.float32)
            images = np.multiply(images, 1.0 / 255.0)
        # 设置私有属性
        self._num_examples = images.shape[0]
        self._images = images
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0

    @property
    def images(self):
        return self._images

    @property
    def labels(self):
        return self._labels

    @property
    def num_examples(self):
        return self.num_examples

    @property
    def epochs_completed(self):
        self._epochs_completed

    # 批量获取训练数据
    def next_batch(self, batch_size,shuffle=True):
        start = self._index_in_epoch
        if self._epochs_completed == 0 and start == 0 and shuffle:
            # 打乱顺序
            perm0 = np.arange(self._num_examples)
            np.random.shuffle(perm0)
            self._images = self._images[perm0]
            self._labels = self._labels[perm0]

        if start + batch_size > self._num_examples:
            self._epochs_completed += 1
            rest_num_examples = self._num_examples - start
            images_rest_part = self._images[start:self._num_examples]
            labels_rest_part = self._labels[start:self._num_examples]
            # 当剩余的数据不够一次batch_size，就在之前的数据中随机选取并进行组合
            if shuffle:
                perm = np.arange(self._num_examples)
                np.random.shuffle(perm)
                self._images = self._images[perm]
                self._labels = self._labels[perm]
            start = 0
            self._index_in_epoch = batch_size - rest_num_examples
            end = self._index_in_epoch
            images_new_part = self._images[start:end]
            labels_new_part = self._labels[start:end]
            return np.concatenate((images_rest_part, images_new_part), axis=0), np.concatenate(
                (labels_rest_part, labels_new_part), axis=0)
        else:
            self._index_in_epoch += batch_size
            end = self._index_in_epoch
            return self._images[start:end], self._labels[start:end]


def input_data(train_dir, dtype = dtypes.float32, reshape = True, seed=None):
    training_size = 28709
    validation_size = 3589
    test_size = 3589

    train_faces, train_emotions = load_data(train_dir)
    print("Data load success!")

    # 验证数据
    validation_faces = train_faces[training_size: training_size + validation_size]
    validation_emotions = train_emotions[training_size: training_size + validation_size]

    # 测试数据
    test_faces = train_faces[training_size + validation_size:]
    test_emotions = train_emotions[training_size + validation_size:]

    # 训练数据
    train_faces = train_faces[: training_size]
    train_emotions = train_emotions[: training_size]

    Datasets = collections.namedtuple('Datasets', ['train', 'validation', 'test'])
    train = DataSet(train_faces, train_emotions, reshape=reshape,)
    validation = DataSet(validation_faces, validation_emotions, dtype=dtype, reshape=reshape, seed=seed)
    test = DataSet(test_faces, test_emotions, dtype=dtype, reshape=reshape, seed=seed)
    return Datasets(train=train, validation=validation, test=test)

In [ ]:
!pip install tensorflow.compat.v1

ERROR: Could not find a version that satisfies the requirement tensorflow.compat.v1 (from versions: none)
ERROR: No matching distribution found for tensorflow.compat.v1


In [ ]:
import os
import cv2
import tensorflow as tf
import tensorflow.compat.v1 as tf


EMOTIONS = ['angry', 'disgusted', 'fearful', 'happy', 'sad', 'surprised', 'neutral']


def deepnn(x):
    x_image = tf.reshape(x, [-1, 48, 48, 1])
    # conv1
    W_conv1 = weight_variables([5, 5, 1, 64])
    b_conv1 = bias_variable([64])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    # pool1
    h_pool1 = maxpool(h_conv1)
    # norm1
    norm1 = tf.nn.lrn(h_pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

    # conv2
    W_conv2 = weight_variables([3, 3, 64, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    norm2 = tf.nn.lrn(h_conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    h_pool2 = maxpool(norm2)

    # Fully connected layer
    W_fc1 = weight_variables([12 * 12 * 64, 384])
    b_fc1 = bias_variable([384])
    h_conv3_flat = tf.reshape(h_pool2, [-1, 12 * 12 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv3_flat, W_fc1) + b_fc1)

    # Fully connected layer
    W_fc2 = weight_variables([384, 192])
    b_fc2 = bias_variable([192])
    h_fc2 = tf.matmul(h_fc1, W_fc2) + b_fc2

    # linear
    W_fc3 = weight_variables([192, 7])
    b_fc3 = bias_variable([7])
    y_conv = tf.add(tf.matmul(h_fc2, W_fc3), b_fc3)

    return y_conv


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def maxpool(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variables(shape):
    initial = tf.random.truncated_normal(shape=shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def train_model(train_data):
    fer2013 = input_data(train_data)
    max_train_steps = 30001

    tf.disable_v2_behavior()

    x = tf.placeholder(tf.float32, [None, 2304])
    y_ = tf.placeholder(tf.float32, [None, 7])

    y_conv = deepnn(x)

    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with tf.Session() as sess:
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())

        for step in range(max_train_steps):
            batch = fer2013.train.next_batch(25)
            if step % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
                print('step %d, training accuracy %g' % (step, train_accuracy))
            if step + 1 == max_train_steps:
                saver.save(sess, '/content/drive/MyDrive/COSC5470/model/emotion_DNN_model', global_step=step + 1)
            train_step.run(feed_dict={x: batch[0], y_: batch[1]})


def image_to_tensor(image):
    tensor = np.asarray(image).reshape(-1, 2304) * 1 / 255.0
    return tensor

In [ ]:
train_data = "/content/drive/MyDrive/COSC5470/challenges-in-representation-learning-facial-expression-recognition-challenge/fer2013.csv"
train_model(train_data)

Data load success!
step 0, training accuracy 0.2
step 100, training accuracy 0.12
step 200, training accuracy 0.12
step 300, training accuracy 0.68
step 400, training accuracy 0.28
step 500, training accuracy 0.48
step 600, training accuracy 0.16
step 700, training accuracy 0.12
step 800, training accuracy 0.4
step 900, training accuracy 0.28
step 1000, training accuracy 0.56
step 1100, training accuracy 0.32
step 1200, training accuracy 0.6
step 1300, training accuracy 0.48
step 1400, training accuracy 0.52
step 1500, training accuracy 0.48
step 1600, training accuracy 0.28
step 1700, training accuracy 0.24
step 1800, training accuracy 0.52
step 1900, training accuracy 0.56
step 2000, training accuracy 0.44
step 2100, training accuracy 0.56
step 2200, training accuracy 0.28
step 2300, training accuracy 0.56
step 2400, training accuracy 0.48
step 2500, training accuracy 0.4
step 2600, training accuracy 0.4
step 2700, training accuracy 0.28
step 2800, training accuracy 0.48
step 2900, t

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import numpy as np
import sys
import tensorflow as tf
import PIL.Image as Image
import matplotlib.pyplot as plt

# 加载opencv自带的人脸识别器
cascade_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 人脸七种微表情
EMOTIONS = ['angry', 'disgusted', 'fearful', 'happy', 'sad', 'surprised', 'neutral']


def format_image(image):
    # image如果为彩色图：image.shape[0][1][2](水平、垂直像素、通道数)
    if len(image.shape) > 2 and image.shape[2] == 3:
        # 将图片变为灰度图
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # 它可以检测出图片中所有的人脸，并将人脸用vector保存各个人脸的坐标、大小（用矩形表示）
        # 调整scaleFactor参数的大小，可以增加识别的灵敏度，推荐1.1
        faces = cascade_classifier.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)
    # 如果图片中没有检测到人脸，则返回None
    if not len(faces) > 0:
        return None, None
    # max_are_face包含了人脸的坐标，大小
    max_are_face = faces[0]
    # 在所有人脸中选一张最大的脸
    for face in faces:
        if face[2] * face[3] > max_are_face[2] * max_are_face[3]:
            max_are_face = face

    # 这两步可有可无
    face_coor = max_are_face
    image = image[face_coor[1]:(face_coor[1] + face_coor[2]), face_coor[0]:(face_coor[0] + face_coor[3])]
    # 调整图片大小，变为48*48
    try:
        image = cv2.resize(image, (48, 48), interpolation=cv2.INTER_CUBIC)
    except Exception:
        print("problem during resize")
        return None, None

    return image, face_coor

In [ ]:
def demo(modelPath, showBox=True):
    # 调用模型分析人脸微表情
#    tf.reset_default_graph()
    # face_x = tf.placeholder(tf.float32, [None, 2304])
    data = np.random.randn(10, 2304).astype(np.float32)

    # Convert numpy array to TensorFlow tensor
    face_x = tf.convert_to_tensor(data)
    y_conv = deepnn(face_x)
    probs = tf.nn.softmax(y_conv)

    # 加载模型
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(modelPath)
    sess = tf.Session()
    print(ckpt == None)
    if ckpt and ckpt.model_checkpoint_path:
      saver.restore(sess, ckpt.model_checkpoint_path)
      print("Restore model sucsses!!\nNOTE: Press 'a' on keyboard to capture face.")

    # feelings_facesy用来存储emojis表情
    feelings_faces = []
    for index, emotion in enumerate(EMOTIONS):
        # imread函数(文件路径,读取方式)
        # cv2.IMREAD_COLOR：读入一副彩色图片；(1)返回三维矩阵，且为[120,120,3]
        # cv2.IMREAD_GRAYSCALE：以灰度模式读入图片；(0)返回二维矩阵，且为[120,120]
        # cv2.IMREAD_UNCHANGED：读入一幅图片，并包括其alpha通道(-1)返回三维矩阵，且为[120,120,4]
        feelings_faces.append(cv2.imread('/content/drive/MyDrive/COSC5470/emojis' + emotion + '.png', 1))

    # 获取笔记本的摄像头，
    video_captor = cv2.VideoCapture(0)

    emoji_face = []
    result = None
    while True:
        # 获取摄像头的每帧图片，若获得，则ret的值为True,frame就是每一帧的图像，是个三维矩阵
        ret, frame = video_captor.read()

        detected_face, face_coor = format_image(frame)
        if showBox:
            if face_coor is not None:
                # 获取人脸的坐标,并用矩形框出
                [x, y, w, h] = face_coor
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)

        # 每隔10ms刷新一次，并且等当键盘输入a的时候，截取图像，因为是64位系统所以必须要0xFF == ord('a')
        if cv2.waitKey(1) & 0xFF == ord('a'):
            if detected_face is not None:
                cv2.imwrite('a.jpg', detected_face)
                print(detected_face)
                print("获取成功")
                # 将图片变为tensorflow可以接受的格式
                tensor = image_to_tensor(detected_face)
                result = sess.run(probs, feed_dict={face_x: tensor})
                print(result)

        if result is not None:
            for index, emotion in enumerate(EMOTIONS):
                # 将七种微表情的文字添加到图片中
                cv2.putText(frame,emotion,(10,index*20 + 20),cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 1)
                # 将七种微表情的概率用矩形表现出来
                cv2.rectangle(frame,(130, index*20 + 10),(130+int(result[0][index]*100), (index + 1) * 20 + 4), (255, 0, 0), -1)
                # 获取人脸微表情相应的emojis表情
                emoji_face = feelings_faces[np.argmax(result[0])]

            # 将emojis表情添加到图片中的指定位置 方法1：
            frame[200:320, 10:130, :] = emoji_face[:, :, :]
            cv2.imwrite('b.jpg', frame)
            # 将emojis表情添加到图片中的指定位置 方法2：
            # for c in range(0, 1):
            #     frame[200:320, 10:130, c] = emoji_face[:, :, c] * (emoji_face[:, :, 3] / 255.0) + frame[200:320, 10:130, c] * (1.0 - emoji_face[:, :, 3] / 255.0)

        cv2.imshow('face', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    # 释放系统摄像头，关闭窗口
    video_captor.release()
    cv2.destroyAllWindows()

demo("/content/drive/MyDrive/COSC5470/model/")
# demo('/content/drive/MyDrive/COSC5470/models/', True)

True


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
video_captor = cv2.VideoCapture(0)

emoji_face = []
result = None
while True:
    # 获取摄像头的每帧图片，若获得，则ret的值为True,frame就是每一帧的图像，是个三维矩阵
    ret, frame = video_captor.read()
    detected_face, face_coor = format_image(frame)
    if showBox:
        if face_coor is not None:
            # 获取人脸的坐标,并用矩形框出
            [x, y, w, h] = face_coor
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)

AttributeError: 'NoneType' object has no attribute 'shape'